In [1]:
'''Отмечу стразу, этот модуль тут не к месту.Для меня он преследует какие-то туманные цели.Я не обладаю достаточным количеством информации и понимания для чего это все.
Что касается самой обработки данных-я это сделаю. На остальное даже нет желания тратить время без понимания для чего это и зачем.Модулю поставил низкую оценку.
Самого же оценка интересует меньше всего.Для меня важно понимание ,а не оценка. Ноутбук желательно выполнять в бруазере,т.к используются модули,кторые лучше всего
работают в браузере без всяких влосипедов.Установлена ширина ноутбука 70% от максимума ширины страницы,поэтому возможны проблемы с отображением на стандарной ширине.'''

'Отмечу стразу, этот модуль тут не к месту.Для меня он преследует какие-то туманные цели.Я не обладаю достаточным количеством информации и понимания для чего это все.\nЧто касается самой обработки данных-я это сделаю. На остальное даже нет желания тратить время без понимания для чего это и зачем.Модулю поставил низкую оценку.\nСамого же оценка интересует меньше всего.Для меня важно понимание ,а не оценка. Ноутбук желательно выполнять в бруазере,т.к используются модули,кторые лучше всего\nработают в браузере без всяких влосипедов.Установлена ширина ноутбука 70% от максимума ширины страницы,поэтому возможны проблемы с отображением на стандарной ширине.'

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from IPython.core.display import display, HTML
from plotly.offline import iplot, download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)

In [3]:
# загрузим датафрейм
df = pd.read_csv('stud_math.csv')
# посмотрим на название колонок,чтобы не было проблем при обращении
df.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'studytime, granular', 'higher', 'internet', 'romantic', 'famrel',
       'freetime', 'goout', 'health', 'absences', 'score'],
      dtype='object')

In [4]:
# имеется непонятная колонка 'studytime, granular',которой не было в описании датафрейма .Дропнем ее
df = df.drop(['studytime, granular'], axis=1)
df.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout',
       'health', 'absences', 'score'],
      dtype='object')

In [5]:
df.info()  # Обзор

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 29 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   school      395 non-null    object 
 1   sex         395 non-null    object 
 2   age         395 non-null    int64  
 3   address     378 non-null    object 
 4   famsize     368 non-null    object 
 5   Pstatus     350 non-null    object 
 6   Medu        392 non-null    float64
 7   Fedu        371 non-null    float64
 8   Mjob        376 non-null    object 
 9   Fjob        359 non-null    object 
 10  reason      378 non-null    object 
 11  guardian    364 non-null    object 
 12  traveltime  367 non-null    float64
 13  studytime   388 non-null    float64
 14  failures    373 non-null    float64
 15  schoolsup   386 non-null    object 
 16  famsup      356 non-null    object 
 17  paid        355 non-null    object 
 18  activities  381 non-null    object 
 19  nursery     379 non-null    o

In [6]:
'''визуализируем по каждому столбцу гистограмму содержащихся типов'''
number = len(df.columns)
fig = make_subplots(rows=1, cols=number, column_titles=list(df.columns))
fig.update_layout(width=2500, height=800, showlegend=False)
objects = [go.Histogram(x=df[columns].apply(lambda x: str(type(x))), name=columns)
           for columns in df.columns]
fig.add_traces(objects,
               rows=[1]*len(df.columns),
               cols=[i for i in range(1, number+1, 1)])

In [7]:
# Заменим NAN  и пропуски в объектных типах на None
for col in df.columns:
    if df[col].dtype == type(object()):
        df[col] = df[col].apply(lambda x: x if isinstance(x, str) else '')
        df[col] = df[col].apply(lambda x: None if x.strip() == '' else x)

In [8]:
# глянем целиком что получилсоь
# по факту реальных числовых значений только 3-возраст,пропуски и оценка. Остальные категориальные,хоть многие и используют числа для маркировки
display(HTML(df.to_html()))

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,health,absences,score
0,GP,F,18,U,None,A,4.0,4.0,at_home,teacher,course,mother,2.0,2.0,0.0,yes,no,no,no,yes,yes,None,no,4.0,3.0,4.0,3.0,6.0,30.0
1,GP,F,17,U,GT3,None,1.0,1.0,at_home,other,course,father,1.0,2.0,0.0,no,yes,no,no,no,yes,yes,no,5.0,3.0,3.0,3.0,4.0,30.0
2,GP,F,15,U,LE3,T,1.0,1.0,at_home,other,other,mother,1.0,2.0,3.0,yes,no,None,no,yes,yes,yes,None,4.0,3.0,2.0,3.0,10.0,50.0
3,GP,F,15,U,GT3,T,4.0,2.0,health,None,home,mother,1.0,3.0,0.0,no,yes,yes,yes,yes,yes,yes,yes,3.0,2.0,2.0,5.0,2.0,75.0
4,GP,F,16,U,GT3,T,3.0,3.0,other,other,home,father,1.0,2.0,0.0,no,yes,yes,no,yes,yes,no,no,4.0,3.0,2.0,5.0,4.0,50.0
5,GP,M,16,U,LE3,T,4.0,3.0,services,other,reputation,mother,1.0,2.0,0.0,no,yes,yes,yes,yes,yes,yes,no,5.0,4.0,2.0,5.0,10.0,75.0
6,GP,M,16,None,LE3,T,2.0,2.0,other,other,home,mother,1.0,2.0,0.0,no,no,no,no,yes,yes,yes,no,4.0,4.0,4.0,3.0,0.0,55.0
7,GP,F,17,U,GT3,A,4.0,4.0,other,teacher,home,mother,2.0,2.0,0.0,yes,yes,no,no,yes,yes,no,no,4.0,1.0,4.0,1.0,6.0,30.0
8,GP,M,15,U,LE3,A,3.0,2.0,services,other,home,mother,1.0,2.0,0.0,no,yes,yes,no,yes,yes,yes,no,NaN,2.0,2.0,1.0,0.0,95.0
9,GP,M,15,U,None,None,3.0,4.0,other,other,home,mother,1.0,2.0,0.0,no,yes,yes,yes,yes,yes,yes,no,5.0,5.0,1.0,5.0,0.0,75.0


In [9]:
'''построим распеределние нечисловых значений'''
objects = [go.Histogram(x=df[columns],
                        name=columns) for columns in
           df.columns if df[columns].dtype == type(object)]
fig = make_subplots(rows=1, cols=len(objects), subplot_titles=[columns for columns in df.columns if df[columns].
                                                               dtype == type(object())])
fig.update_layout(height=800, width=1600, showlegend=False)
fig.add_traces(objects,
               rows=[1]*len(objects),
               cols=[i for i in range(1, len(objects)+1, 1)])

In [10]:
'''строим распеределение признаков для столбцов с категориальными данными,обозначенными числом значениями(сюда же
отнесем и возраст,хотя он и не совсем категориальный признак)'''
names = [columns for columns in df.columns if (
    (df[columns].dtype != type(object())) & (columns not in ['absences', 'score']))]
objects = [go.Box(y=df[columns], name=columns) for columns in df.columns if (
    (df[columns].dtype != type(object())) & (columns not in ['absences', 'score']))]
fig = make_subplots(rows=1, cols=len(objects), column_titles=names)
fig.update_layout(height=800, width=1600, showlegend=False)
fig.add_traces(objects,
               rows=[1]*len(objects),
               cols=[i for i in range(1, len(objects)+1, 1)])

In [11]:
'''теперь строим распеределение признаков для столбцов с истинными числовыми данными'''
names = [columns for columns in df.columns if (
    (df[columns].dtype != type(object())) & (columns in ['absences', 'score']))]
objects = [go.Box(y=df[columns], name=columns) for columns in df.columns if (
    (df[columns].dtype != type(object())) & (columns in ['absences', 'score']))]
fig = make_subplots(rows=1, cols=len(objects), column_titles=names)
fig.update_layout(height=800, width=1600, showlegend=False)
fig.add_traces(objects,
               rows=[1]*len(objects),
               cols=[i for i in range(1, len(objects)+1, 1)])

In [12]:
"теперь оценим графики и посмотрим что необходимо сделать"
# 2.Absences-действительно много серьезных выбросов,что скорее указывает на ошибку,т.к к примеру врядли человек с пропусками 385 дней набрал бы 85 баллов
# 17 выбросов,думаю стоит их убрать
display(df['absences'][df['absences'] > 20].value_counts().sum())
# 3.Тут тоже не ясно.Мне кажется 0 балов невозможно набрать.сколько таких?
# 37 человек.Не многовато ли?думаю стоит выкинуть
display(df['score'][df['score'] == 0].value_counts())
df['absences'] = df['absences'].apply(
    lambda x: np.nan if x > 20 else x)  # заменим их на NaN
df['score'] = df['score'].apply(lambda x: np.nan if x == 0 else x)
df = df.dropna(subset=['absences', 'score'], axis=0)  # и дропнем все NaN

17

0.0    37
Name: score, dtype: int64

In [23]:
fig = px.histogram(df['score'][df['score'] != 0])
fig.update_layout(height=800, width=800)
fig.show()  # так получше

In [17]:
# корреляции между числовыми признаками не заметно
df[['absences', 'score']].corr()

,absences,score
absences,1.00000,-0.24222
score,-0.24222,1.00000


In [24]:
fig = px.scatter_matrix(df[['absences', 'score']])
fig.update_layout(width=800, height=800)

In [ ]:
'''В заключение отмечу,из того, что пытались сделать при анализе номинативных переменных я не понял ровным счетом ничего.
То объяснение которое приводится не выдерживает никакой критики'''